In [1]:
import os
import json
import time
import random
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [3]:
!nvidia-smi

Thu Apr  6 22:55:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 24%   57C    P2    70W / 250W |   5517MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
config = dict(
    saved_path="saved_models/random.pt",
    best_saved_path = "saved/random_best.pt",
    lr=0.001, 
    EPOCHS = 3,
    BATCH_SIZE = 32,
    IMAGE_SIZE = 224,
    TRAIN_VALID_SPLIT = 0.2,
    device=device,
    SEED = 42,
    pin_memory=True,
    num_workers=2,
    USE_AMP = True,
    channels_last=False)

random.seed(config['SEED'])
# If you or any of the libraries you are using rely on NumPy, you can seed the global NumPy RNG 
np.random.seed(config['SEED'])
# Prevent RNG for CPU and GPU using torch
torch.manual_seed(config['SEED'])
torch.cuda.manual_seed(config['SEED'])
torch.backends.cudnn.benchmarks = True
torch.backends.cudnn.deterministic = True

torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

# DATA method (Pre-Processing)

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((config['IMAGE_SIZE'],config['IMAGE_SIZE'])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
# Load data
train_data = torchvision.datasets.CIFAR10(root='../Images', train=True, download=True, transform=data_transforms['train'])
test_data = torchvision.datasets.CIFAR10(root='../Images', train=False, download=True, transform=data_transforms['test'])
valid_data = test_data

train_dl = torch.utils.data.DataLoader(train_data, batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])


test_dl = torch.utils.data.DataLoader(test_data, batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])
valid_dl = torch.utils.data.DataLoader(valid_data, batch_size=32,shuffle=True, num_workers = config['num_workers'],
                                          pin_memory = config['pin_memory'])

Files already downloaded and verified
Files already downloaded and verified


In [8]:
def train_model(model,criterion,optimizer,num_epochs=10):

    since = time.time()                                            
    batch_ct = 0
    example_ct = 0
    best_acc = 0.3
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        run_corrects = 0
        #Training
        model.train()
        for x,y in train_dl: #BS=32 ([BS,3,224,224], [BS,4])            
            if config['channels_last']:
                x = x.to(config['device'], memory_format=torch.channels_last) #CHW --> #HWC
            else:
                x = x.to(config['device'])
            y = y.to(config['device']) #CHW --> #HWC
            
            
            
            optimizer.zero_grad()
            #optimizer.zero_grad(set_to_none=True)
            ######################################################################
            
            train_logits = model(x) #Input = [BS,3,224,224] (Image) -- Model --> [BS,4] (Output Scores)
            
            _, train_preds = torch.max(train_logits, 1)
            train_loss = criterion(train_logits,y)
            train_loss = criterion(train_logits,y)
            run_corrects += torch.sum(train_preds == y.data)
            
            train_loss.backward() # Backpropagation this is where your W_gradient
            loss=train_loss

            optimizer.step() # W_new = W_old - LR * W_gradient 
            example_ct += len(x) 
            batch_ct += 1
            if ((batch_ct + 1) % 400) == 0:
                train_log(loss, example_ct, epoch)
            ########################################################################
        
        #validation
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        total = 0
        # Disable gradient calculation for validation or inference using torch.no_rad()
        with torch.no_grad():
            for x,y in valid_dl:
                if config['channels_last']:
                    x = x.to(config['device'], memory_format=torch.channels_last) #CHW --> #HWC
                else:
                    x = x.to(config['device'])
                y = y.to(config['device'])
                valid_logits = model(x)
                _, valid_preds = torch.max(valid_logits, 1)
                valid_loss = criterion(valid_logits,y)
                running_loss += valid_loss.item() * x.size(0)
                running_corrects += torch.sum(valid_preds == y.data)
                total += y.size(0)
            
        epoch_loss = running_loss / len(valid_data)
        epoch_acc = running_corrects.double() / len(valid_data)
        train_acc = run_corrects.double() / len(train_data)
        print("Train Accuracy",train_acc.cpu())
        print("Validation Loss is {}".format(epoch_loss))
        print("Validation Accuracy is {}\n".format(epoch_acc.cpu()))
        if epoch_acc.cpu()>best_acc:
            print('One of the best validation accuracy found.\n')
            #torch.save(model.state_dict(), config['best_saved_path'])
            best_acc = epoch_acc.cpu()

            
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    torch.save(model.state_dict(), config['saved_path'])

    
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [15]:
efficientnet = models.efficientnet_b0(pretrained = True)
efficientnet.classifier[1] = nn.Linear(in_features = 1280, out_features = 10, bias = True)
model = efficientnet
criterion = nn.CrossEntropyLoss()
model = model.to(config['device'])
optimizer = optim.Adam(model.parameters(),lr=config['lr'])
#model.load_state_dict(torch.load('saved_models/child.pt'))
train_model(model,criterion,optimizer,num_epochs=8)

Epoch 0/7
----------
Loss after 12768 examples: 0.472
Loss after 25568 examples: 0.265
Loss after 38368 examples: 0.423
Train Accuracy tensor(0.8500, dtype=torch.float64)
Validation Loss is 0.28712995879650116
Validation Accuracy is 0.9034000000000001

One of the best validation accuracy found.

Epoch 1/7
----------
Loss after 51152 examples: 0.258
Loss after 63952 examples: 0.161
Loss after 76752 examples: 0.259
Loss after 89552 examples: 0.498
Train Accuracy tensor(0.9102, dtype=torch.float64)
Validation Loss is 0.26927664377093313
Validation Accuracy is 0.9115000000000001

One of the best validation accuracy found.

Epoch 2/7
----------
Loss after 102336 examples: 0.178
Loss after 115136 examples: 0.045
Loss after 127936 examples: 0.186
Loss after 140736 examples: 0.223
Train Accuracy tensor(0.9283, dtype=torch.float64)
Validation Loss is 0.21797222988158466
Validation Accuracy is 0.9293

One of the best validation accuracy found.

Epoch 3/7
----------
Loss after 153520 examples: 0.

In [11]:
efficientnet = models.efficientnet_b0(pretrained = True)
efficientnet.classifier[1] = nn.Linear(in_features = 1280, out_features = 10, bias = True)
model = efficientnet
criterion = nn.CrossEntropyLoss()
model = model.to(config['device'])
optimizer = optim.Adam(model.parameters(),lr=config['lr'])
model.load_state_dict(torch.load('saved_models/random.pt'))

<All keys matched successfully>

In [12]:
def evaluation(model,test_dl):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    preds = []
    pred_labels = []
    labels = []

    with torch.no_grad():
                for x,y in test_dl:
                    x = x.to(config['device'])
                    y = y.to(config['device']) #CHW --> #HWC
                    valid_logits = model(x)
                    predict_prob = F.softmax(valid_logits)
                    _,predictions = predict_prob.max(1)
                    predictions = predictions.to('cpu')

                    _, valid_preds = torch.max(valid_logits, 1)
                    valid_loss = criterion(valid_logits,y)
                    running_loss += valid_loss.item() * x.size(0)
                    running_corrects += torch.sum(valid_preds == y.data)
                    total += y.size(0)
                    predict_prob = predict_prob.to('cpu')

                    pred_labels.extend(list(predictions.numpy()))
                    preds.extend(list(predict_prob.numpy()))
                    y = y.to('cpu')
                    labels.extend(list(y.numpy()))

    epoch_loss = running_loss / len(test_data)
    epoch_acc = running_corrects.double() / len(test_data)
    print("Test Loss is {}".format(epoch_loss))
    print("Test Accuracy is {}".format(epoch_acc.cpu()))
    return np.array(labels),np.array(pred_labels),np.array(preds)
    

labels, pred_labels,preds = evaluation(model, test_dl)
#print(metrics.precision_recall_fscore_support(np.array(labels), np.array(pred_labels)))
print('\nAUROC:')
print(metrics.roc_auc_score(np.array(labels), np.array(preds), multi_class='ovr'))
print(metrics.classification_report(labels,pred_labels))

Test Loss is 0.23503911691755056
Test Accuracy is 0.9334

AUROC:
0.9970546777777777
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      1000
           1       0.95      0.97      0.96      1000
           2       0.93      0.90      0.92      1000
           3       0.90      0.82      0.86      1000
           4       0.96      0.93      0.94      1000
           5       0.87      0.91      0.89      1000
           6       0.92      0.97      0.94      1000
           7       0.96      0.97      0.97      1000
           8       0.97      0.96      0.96      1000
           9       0.96      0.96      0.96      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [11]:
def calculate_disparate_impact(model, dataloader):
    #device = next(model.parameters()).device
    model.eval()
    with torch.no_grad():
        class_counts = torch.zeros((10,))
        correct_counts = torch.zeros((10,))
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            for i in range(10):
                class_mask = labels == i
                class_counts[i] += torch.sum(class_mask)
                correct_counts[i] += torch.sum(preds[class_mask] == labels[class_mask])

        proportions = correct_counts / class_counts
        reference_proportion = torch.mean(proportions)
        max_proportion = torch.max(proportions)
        
        return reference_proportion / max_proportion
    
device = 'cpu'
model = model.to('cpu')
disparate_impact = calculate_disparate_impact(model, test_dl)
print(f"Disparate Impact: {disparate_impact:.4f}")

Disparate Impact: 0.9573


# ALGORITHMIC method

In [11]:
class ReweightedCELoss(torch.nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()
        self.num_classes = num_classes
        self.device = device
        self.class_counts = torch.zeros((self.num_classes,)).to(self.device)
        self.loss_weights = torch.ones((self.num_classes,)).to(self.device)

    def forward(self, logits, labels):
        batch_size = logits.size(0)

        # Compute standard cross-entropy loss
        loss_ce = F.cross_entropy(logits, labels)

        # Update class counts and loss weights
        class_counts_batch = torch.zeros((self.num_classes,)).to(self.device)
        for i in range(batch_size):
            class_counts_batch[labels[i]] += 1
        self.class_counts += class_counts_batch
        self.loss_weights = 1 / torch.sqrt(self.class_counts)

        # Compute re-weighted loss
        loss_rce = 0
        for c in range(self.num_classes):
            mask_c = labels == c
            if torch.sum(mask_c) == 0:
                continue
            logits_c = logits[:, c]
            loss_weights_c = self.loss_weights[c]
            loss_rce += torch.mean(loss_weights_c * F.binary_cross_entropy_with_logits(logits_c, mask_c.float()))

        return loss_ce + loss_rce

criterion = ReweightedCELoss(num_classes=10, device=device)

efficientnet = models.efficientnet_b0(pretrained = True)
efficientnet.classifier[1] = nn.Linear(in_features = 1280, out_features = 10, bias = True)
model = efficientnet
model = model.to(config['device'])
optimizer = optim.Adam(model.parameters(),lr=config['lr'])
train_model(model,criterion,optimizer,num_epochs=8)

Epoch 0/7
----------
Loss after 12768 examples: 0.695
Loss after 25568 examples: 0.564
Loss after 38368 examples: 0.446
Train Accuracy tensor(0.8496, dtype=torch.float64)
Validation Loss is 0.2618277025580406
Validation Accuracy is 0.9157000000000001

One of the best validation accuracy found.

Epoch 1/7
----------
Loss after 51152 examples: 0.147
Loss after 63952 examples: 0.244
Loss after 76752 examples: 0.176
Loss after 89552 examples: 0.076
Train Accuracy tensor(0.9090, dtype=torch.float64)
Validation Loss is 0.2631435434222221
Validation Accuracy is 0.9158000000000001

One of the best validation accuracy found.

Epoch 2/7
----------
Loss after 102336 examples: 0.044
Loss after 115136 examples: 0.248
Loss after 127936 examples: 0.026
Loss after 140736 examples: 0.260
Train Accuracy tensor(0.9266, dtype=torch.float64)
Validation Loss is 0.2133455213084817
Validation Accuracy is 0.9285

One of the best validation accuracy found.

Epoch 3/7
----------
Loss after 153520 examples: 0.059

In [12]:
def evaluation(model,test_dl):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    preds = []
    pred_labels = []
    labels = []

    with torch.no_grad():
                for x,y in test_dl:
                    x = x.to(config['device'])
                    y = y.to(config['device']) #CHW --> #HWC
                    valid_logits = model(x)
                    predict_prob = F.softmax(valid_logits)
                    _,predictions = predict_prob.max(1)
                    predictions = predictions.to('cpu')

                    _, valid_preds = torch.max(valid_logits, 1)
                    valid_loss = criterion(valid_logits,y)
                    running_loss += valid_loss.item() * x.size(0)
                    running_corrects += torch.sum(valid_preds == y.data)
                    total += y.size(0)
                    predict_prob = predict_prob.to('cpu')

                    pred_labels.extend(list(predictions.numpy()))
                    preds.extend(list(predict_prob.numpy()))
                    y = y.to('cpu')
                    labels.extend(list(y.numpy()))

    epoch_loss = running_loss / len(test_data)
    epoch_acc = running_corrects.double() / len(test_data)
    print("Test Loss is {}".format(epoch_loss))
    print("Test Accuracy is {}".format(epoch_acc.cpu()))
    return np.array(labels),np.array(pred_labels),np.array(preds)
    

labels, pred_labels,preds = evaluation(model, test_dl)
#print(metrics.precision_recall_fscore_support(np.array(labels), np.array(pred_labels)))
print('\nAUROC:')
print(metrics.roc_auc_score(np.array(labels), np.array(preds), multi_class='ovr'))
print(metrics.classification_report(labels,pred_labels))

Test Loss is 0.23201741560399533
Test Accuracy is 0.9345

AUROC:
0.9968824833333333
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1000
           1       0.98      0.94      0.96      1000
           2       0.93      0.91      0.92      1000
           3       0.89      0.84      0.87      1000
           4       0.91      0.95      0.93      1000
           5       0.92      0.90      0.91      1000
           6       0.95      0.96      0.95      1000
           7       0.97      0.96      0.97      1000
           8       0.96      0.95      0.96      1000
           9       0.93      0.98      0.95      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [14]:
def calculate_disparate_impact(model, dataloader):
    #device = next(model.parameters()).device
    model.eval()
    with torch.no_grad():
        class_counts = torch.zeros((10,))
        correct_counts = torch.zeros((10,))
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            for i in range(10):
                class_mask = labels == i
                class_counts[i] += torch.sum(class_mask)
                correct_counts[i] += torch.sum(preds[class_mask] == labels[class_mask])

        proportions = correct_counts / class_counts
        reference_proportion = torch.mean(proportions)
        max_proportion = torch.max(proportions)
        return reference_proportion / max_proportion
    
device = 'cpu'
model = model.to('cpu')
disparate_impact = calculate_disparate_impact(model, test_dl)
print(f"Disparate Impact: {disparate_impact:.4f}")

Disparate Impact: 0.9565
